In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np 
import pandas as pd 
import string 
import spacy 

from matplotlib.pyplot import imread
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS

In [6]:
ts=pd.read_csv('../input/elon-musk/Elon_musk.csv',encoding='latin1')

In [7]:
ts.drop(['Unnamed: 0'],inplace=True,axis=1)
ts

In [8]:
ts.info()

In [9]:
ts=[Text.strip() for Text in ts.Text]
ts=[Text for Text in ts if Text]
ts[0:10]

In [10]:
ts_text=' '.join(ts)
ts_text

In [11]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer(strip_handles=True)
ts_tokens=tknzr.tokenize(ts_text)
print(ts_tokens)

In [12]:
ts_tokens_text=' '.join(ts_tokens)
ts_tokens_text

In [13]:
no_punc_text=ts_tokens_text.translate(str.maketrans('','',string.punctuation))
no_punc_text

In [14]:
import re
no_url_text=re.sub(r'http\S+', '', no_punc_text)
no_url_text

In [15]:
import nltk
nltk.download('punkt')

In [16]:
from nltk.tokenize import word_tokenize
text_tokens=word_tokenize(no_url_text)
print(text_tokens)

In [17]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

In [18]:
stop =pd.read_csv("../input/stopfile/stop.txt")

In [19]:
stop

In [20]:
stop.info()

In [21]:
from nltk.corpus import stopwords
my_stop_words=stopwords.words('english')

stoplist=stop.values.tolist()
sw_list = ['\x92','rt','ye','yeah','haha','Yes','U0001F923','I']
my_stop_words.extend(sw_list)

no_stop_tokens=[word for word in text_tokens if not word in my_stop_words]
print(no_stop_tokens)

In [22]:
stoplist

In [23]:
no_stop_tokens=[word for word in text_tokens if not word in stoplist]
print(no_stop_tokens)

In [24]:
lower_words=[Text.lower() for Text in no_stop_tokens]
print(lower_words[100:200])

In [25]:
from nltk.stem import PorterStemmer
ps=PorterStemmer()
stemmed_tokens=[ps.stem(word) for word in lower_words]
print(stemmed_tokens[100:200])

In [26]:
nlp=spacy.load('en_core_web_sm')
doc=nlp(' '.join(lower_words))
print(doc)

In [27]:
lemmas=[token.lemma_ for token in doc]
print(lemmas)

In [28]:
clean_data=' '.join(lemmas)
clean_data

In [29]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
tscv=cv.fit_transform(lemmas)

In [31]:
print(cv.get_feature_names()[100:200])

In [32]:
print(tscv.toarray()[100:200])

In [33]:
print(tscv.toarray().shape)

In [34]:
cv_ngram_range=CountVectorizer(analyzer='word',ngram_range=(1,3),max_features=100)
bow_matrix_ngram=cv_ngram_range.fit_transform(lemmas)

In [35]:
print(cv_ngram_range.get_feature_names())
print(bow_matrix_ngram.toarray())

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfv_ngram_max_features=TfidfVectorizer(norm='l2',analyzer='word',ngram_range=(1,3),max_features=500)
tfidf_matix_ngram=tfidfv_ngram_max_features.fit_transform(lemmas)

In [37]:
print(tfidfv_ngram_max_features.get_feature_names())
print(tfidf_matix_ngram.toarray())

In [38]:
def plot_cloud(wordcloud):
    plt.figure(figsize=(40,30))
    plt.imshow(wordcloud)
    plt.axis('off')

In [39]:
STOPWORDS.add('pron')
STOPWORDS.add('rt')
STOPWORDS.add('yeah')

In [40]:
wordcloud=WordCloud(width=3000,height=2000,background_color='black',max_words=50,
                    colormap='Set1',stopwords=STOPWORDS).generate(clean_data)

In [41]:
plot_cloud(wordcloud)

In [42]:
nlp=spacy.load('en_core_web_sm')

one_block=clean_data
doc_block=nlp(one_block)
spacy.displacy.render(doc_block,style='ent',jupyter=True)

In [43]:
for token in doc_block[100:200]:
    print(token,token.pos_) 

In [44]:
nouns_verbs=[token.text for token in doc_block if token.pos_ in ('NOUN','VERB')]
print(nouns_verbs[100:200])

In [45]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()

X=cv.fit_transform(nouns_verbs)
sum_words=X.sum(axis=0)

words_freq=[(word,sum_words[0,idx]) for word,idx in cv.vocabulary_.items()]
words_freq=sorted(words_freq, key=lambda x: x[1], reverse=True)

wd_df=pd.DataFrame(words_freq)
wd_df.columns=['word','count']
wd_df[0:10] 

In [46]:
wd_df[0:10].plot.bar(x='word',figsize=(12,8),title='Top 10 nouns and verbs');

In [47]:
from nltk import tokenize
sentences=tokenize.sent_tokenize(' '.join(ts))
sentences

In [48]:
sent_df=pd.DataFrame(sentences,columns=['sentence'])
sent_df

In [50]:
affin=pd.read_csv("../input/affine123/Afinn.csv",sep=',',encoding='Latin-1')
affin

In [51]:
affinity_scores=affin.set_index('word')['value'].to_dict()
affinity_scores

In [52]:
nlp=spacy.load('en_core_web_sm')
sentiment_lexicon=affinity_scores

def calculate_sentiment(text:str=None):
    sent_score=0
    if text:
        sentence=nlp(text)
        for word in sentence:
            sent_score+=sentiment_lexicon.get(word.lemma_,0)
    return sent_score

In [53]:
calculate_sentiment(text='great')

In [54]:
sent_df['sentiment_value']=sent_df['sentence'].apply(calculate_sentiment)
sent_df['sentiment_value']

In [55]:
sent_df['word_count']=sent_df['sentence'].str.split().apply(len)
sent_df['word_count']

In [56]:
sent_df.sort_values(by='sentiment_value')

In [57]:
sent_df['sentiment_value'].describe()

In [58]:
sent_df[sent_df['sentiment_value']<=0]

In [59]:
sent_df[sent_df['sentiment_value']>0]

In [60]:
sent_df['index']=range(0,len(sent_df))
sent_df

In [61]:
import seaborn as sns
plt.figure(figsize=(15,10))
sns.distplot(sent_df['sentiment_value'])

In [62]:
sent_df.plot.scatter(x='word_count',y='sentiment_value',figsize=(8,8),title='Sentence sentiment value to sentence word count')

In [63]:
plt.figure(figsize=(15,10))
sns.lineplot(y='sentiment_value',x='index',data=sent_df)